# ਸੈਮੈਂਟਿਕ ਕਰਨਲ ਨਾਲ OpenBnB MCP ਸਰਵਰ ਇੰਟੀਗ੍ਰੇਸ਼ਨ

ਇਹ ਨੋਟਬੁੱਕ ਦਿਖਾਉਂਦੀ ਹੈ ਕਿ ਸੈਮੈਂਟਿਕ ਕਰਨਲ ਨੂੰ ਅਸਲ OpenBnB MCP ਸਰਵਰ ਨਾਲ ਵਰਤ ਕੇ MCPStdioPlugin ਦੀ ਮਦਦ ਨਾਲ ਅਸਲ Airbnb ਰਿਹਾਇਸ਼ਾਂ ਦੀ ਖੋਜ ਕਿਵੇਂ ਕੀਤੀ ਜਾ ਸਕਦੀ ਹੈ। LLM ਐਕਸੈਸ ਲਈ, ਇਹ Azure AI Foundry ਵਰਤਦੀ ਹੈ। ਆਪਣੇ ਵਾਤਾਵਰਣ ਵੈਰੀਏਬਲ ਸੈਟਅੱਪ ਕਰਨ ਲਈ, ਤੁਸੀਂ [ਸੈਟਅੱਪ ਪਾਠ]( /00-course-setup/README.md) ਦੀ ਪਾਲਣਾ ਕਰ ਸਕਦੇ ਹੋ।


## ਲੋੜੀਂਦੇ ਪੈਕੇਜਾਂ ਨੂੰ ਇੰਪੋਰਟ ਕਰੋ


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP ਪਲੱਗਇਨ ਕਨੈਕਸ਼ਨ ਬਣਾਉਣਾ

ਅਸੀਂ MCPStdioPlugin ਦੀ ਵਰਤੋਂ ਕਰਕੇ [OpenBnB MCP ਸਰਵਰ](https://github.com/openbnb-org/mcp-server-airbnb) ਨਾਲ ਕਨੈਕਟ ਕਰਾਂਗੇ। ਇਹ ਸਰਵਰ @openbnb/mcp-server-airbnb ਪੈਕੇਜ ਰਾਹੀਂ Airbnb ਖੋਜ ਦੀ ਸਹੂਲਤ ਪ੍ਰਦਾਨ ਕਰਦਾ ਹੈ।


## ਕਲਾਇੰਟ ਬਣਾਉਣਾ

ਇਸ ਉਦਾਹਰਨ ਵਿੱਚ, ਅਸੀਂ ਆਪਣੇ LLM ਐਕਸੈੱਸ ਲਈ Azure AI Foundry ਦੀ ਵਰਤੋਂ ਕਰਾਂਗੇ। ਯਕੀਨੀ ਬਣਾਓ ਕਿ ਤੁਹਾਡੇ ਵਾਤਾਵਰਣ ਵੈਰੀਏਬਲ ਸਹੀ ਤਰੀਕੇ ਨਾਲ ਸੈਟ ਕੀਤੇ ਗਏ ਹਨ।


## ਪਰਿਵਰਤਨ ਵਾਤਾਵਰਣ

Azure OpenAI ਸੈਟਿੰਗਾਂ ਨੂੰ ਕਨਫਿਗਰ ਕਰੋ। ਯਕੀਨੀ ਬਣਾਓ ਕਿ ਤੁਹਾਡੇ ਕੋਲ ਹੇਠ ਲਿਖੀਆਂ ਵਾਤਾਵਰਣ ਵੈਰੀਏਬਲ ਸੈਟ ਕੀਤੀਆਂ ਹਨ:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP ਇੰਟੀਗ੍ਰੇਸ਼ਨ ਨੂੰ ਸਮਝਣਾ

ਇਹ ਨੋਟਬੁੱਕ **ਅਸਲੀ OpenBnB MCP ਸਰਵਰ** ਨਾਲ ਜੁੜਦੀ ਹੈ ਜੋ ਅਸਲੀ Airbnb ਖੋਜ ਦੀ ਸਹੂਲਤ ਪ੍ਰਦਾਨ ਕਰਦੀ ਹੈ।

### ਇਹ ਕਿਵੇਂ ਕੰਮ ਕਰਦਾ ਹੈ:

1. **MCPStdioPlugin**: MCP ਸਰਵਰ ਨਾਲ ਸਧਾਰਨ ਇਨਪੁਟ/ਆਉਟਪੁੱਟ ਸੰਚਾਰ ਦੀ ਵਰਤੋਂ ਕਰਦਾ ਹੈ
2. **ਅਸਲੀ NPM ਪੈਕੇਜ**: `@openbnb/mcp-server-airbnb` ਨੂੰ npx ਰਾਹੀਂ ਡਾਊਨਲੋਡ ਅਤੇ ਚਲਾਉਂਦਾ ਹੈ
3. **ਲਾਈਵ ਡਾਟਾ**: Airbnb ਦੇ APIs ਤੋਂ ਅਸਲੀ ਜਾਇਦਾਦਾਂ ਦੇ ਡਾਟਾ ਨੂੰ ਵਾਪਸ ਕਰਦਾ ਹੈ
4. **ਫੰਕਸ਼ਨ ਖੋਜ**: ਏਜੰਟ MCP ਸਰਵਰ ਤੋਂ ਉਪਲਬਧ ਫੰਕਸ਼ਨਾਂ ਨੂੰ ਆਪਣੇ ਆਪ ਖੋਜ ਲੈਂਦਾ ਹੈ

### ਉਪਲਬਧ ਫੰਕਸ਼ਨ:

OpenBnB MCP ਸਰਵਰ ਆਮ ਤੌਰ 'ਤੇ ਇਹ ਪ੍ਰਦਾਨ ਕਰਦਾ ਹੈ:
- **search_listings** - ਸਥਾਨ ਅਤੇ ਮਾਪਦੰਡਾਂ ਦੁਆਰਾ Airbnb ਜਾਇਦਾਦਾਂ ਦੀ ਖੋਜ ਕਰੋ
- **get_listing_details** - ਖਾਸ ਜਾਇਦਾਦਾਂ ਬਾਰੇ ਵਿਸਥਾਰਿਤ ਜਾਣਕਾਰੀ ਪ੍ਰਾਪਤ ਕਰੋ
- **check_availability** - ਖਾਸ ਤਾਰੀਖਾਂ ਲਈ ਉਪਲਬਧਤਾ ਦੀ ਜਾਂਚ ਕਰੋ
- **get_reviews** - ਜਾਇਦਾਦਾਂ ਲਈ ਸਮੀਖਾਵਾਂ ਪ੍ਰਾਪਤ ਕਰੋ
- **get_host_info** - ਜਾਇਦਾਦ ਦੇ ਹੋਸਟਾਂ ਬਾਰੇ ਜਾਣਕਾਰੀ ਪ੍ਰਾਪਤ ਕਰੋ

### ਪੂਰਵ ਸ਼ਰਤਾਂ:

- **Node.js** ਤੁਹਾਡੇ ਸਿਸਟਮ 'ਤੇ ਇੰਸਟਾਲ ਹੋਣਾ ਚਾਹੀਦਾ ਹੈ
- **ਇੰਟਰਨੈਟ ਕਨੈਕਸ਼ਨ** MCP ਸਰਵਰ ਪੈਕੇਜ ਨੂੰ ਡਾਊਨਲੋਡ ਕਰਨ ਲਈ
- **NPX** ਉਪਲਬਧ ਹੋਣਾ (ਜੋ Node.js ਦੇ ਨਾਲ ਆਉਂਦਾ ਹੈ)

### ਕਨੈਕਸ਼ਨ ਦੀ ਜਾਂਚ:

ਤੁਸੀਂ MCP ਸਰਵਰ ਨੂੰ ਹੱਥੋਂ ਚਲਾਕੇ ਜਾਂਚ ਸਕਦੇ ਹੋ:
```bash
npx -y @openbnb/mcp-server-airbnb
```

ਇਹ OpenBnB MCP ਸਰਵਰ ਨੂੰ ਡਾਊਨਲੋਡ ਅਤੇ ਸ਼ੁਰੂ ਕਰੇਗਾ, ਜਿਸ ਨਾਲ Semantic Kernel ਅਸਲੀ Airbnb ਡਾਟਾ ਲਈ ਜੁੜਦਾ ਹੈ।


## ਓਪਨਬੀਐਨਬੀ ਐਮਸੀਪੀ ਸਰਵਰ ਨਾਲ ਏਜੰਟ ਚਲਾਉਣਾ

ਹੁਣ ਅਸੀਂ AI ਏਜੰਟ ਚਲਾਵਾਂਗੇ ਜੋ ਓਪਨਬੀਐਨਬੀ ਐਮਸੀਪੀ ਸਰਵਰ ਨਾਲ ਜੁੜਦਾ ਹੈ ਤਾਂ ਜੋ 2 ਵੱਡਿਆਂ ਅਤੇ 1 ਬੱਚੇ ਲਈ ਸਟਾਕਹੋਮ ਵਿੱਚ ਅਸਲੀ ਐਰਬੀਐਨਬੀ ਰਹਿਣ ਦੀਆਂ ਥਾਵਾਂ ਦੀ ਖੋਜ ਕਰ ਸਕੇ। ਖੋਜ ਦੇ ਮਾਪਦੰਡਾਂ ਨੂੰ ਬਦਲਣ ਲਈ `user_inputs` ਸੂਚੀ ਨੂੰ ਬਦਲਣ ਲਈ ਬੇਝਿਜਕ ਹੋਵੋ।


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# ਸਾਰ
ਵਧਾਈ ਹੋਵੇ! ਤੁਸੀਂ ਸਫਲਤਾਪੂਰਵਕ ਇੱਕ AI ਏਜੰਟ ਬਣਾਇਆ ਹੈ ਜੋ ਮਾਡਲ ਕਾਂਟੈਕਸਟ ਪ੍ਰੋਟੋਕੋਲ (MCP) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਸਲੀ ਦੁਨੀਆ ਦੇ ਅਕੋਮੋਡੇਸ਼ਨ ਖੋਜ ਨਾਲ ਇੰਟੀਗਰੇਟ ਕਰਦਾ ਹੈ:

## ਵਰਤੀ ਗਈ ਤਕਨਾਲੋਜੀਆਂ:
- ਸੈਮੈਂਟਿਕ ਕਰਨਲ - Azure OpenAI ਨਾਲ ਬੁੱਧੀਮਾਨ ਏਜੰਟ ਬਣਾਉਣ ਲਈ
- Azure AI Foundry - LLM ਸਮਰੱਥਾ ਅਤੇ ਚੈਟ ਪੂਰਨਤਾ ਲਈ
- MCP (ਮਾਡਲ ਕਾਂਟੈਕਸਟ ਪ੍ਰੋਟੋਕੋਲ) - ਸਟੈਂਡਰਡ ਟੂਲ ਇੰਟੀਗ੍ਰੇਸ਼ਨ ਲਈ
- OpenBnB MCP ਸਰਵਰ - ਅਸਲੀ Airbnb ਖੋਜ ਫੰਕਸ਼ਨਲਿਟੀ ਲਈ
- Node.js/NPX - ਬਾਹਰੀ MCP ਸਰਵਰ ਚਲਾਉਣ ਲਈ

## ਤੁਸੀਂ ਕੀ ਸਿੱਖਿਆ:
- MCP ਇੰਟੀਗ੍ਰੇਸ਼ਨ: ਸੈਮੈਂਟਿਕ ਕਰਨਲ ਏਜੰਟ ਨੂੰ ਬਾਹਰੀ MCP ਸਰਵਰਾਂ ਨਾਲ ਕਨੈਕਟ ਕਰਨਾ
- ਰੀਅਲ-ਟਾਈਮ ਡਾਟਾ ਐਕਸੈਸ: ਲਾਈਵ APIs ਰਾਹੀਂ ਅਸਲੀ Airbnb ਪ੍ਰਾਪਰਟੀਆਂ ਦੀ ਖੋਜ
- ਪ੍ਰੋਟੋਕੋਲ ਕਮਿਊਨਿਕੇਸ਼ਨ: ਏਜੰਟ ਅਤੇ MCP ਸਰਵਰ ਦੇ ਵਿਚਕਾਰ stdio ਕਮਿਊਨਿਕੇਸ਼ਨ ਦੀ ਵਰਤੋਂ
- ਫੰਕਸ਼ਨ ਡਿਸਕਵਰੀ: MCP ਸਰਵਰਾਂ ਤੋਂ ਉਪਲਬਧ ਫੰਕਸ਼ਨਾਂ ਨੂੰ ਆਟੋਮੈਟਿਕ ਤਰੀਕੇ ਨਾਲ ਖੋਜਣਾ
- ਸਟ੍ਰੀਮਿੰਗ ਰਿਸਪਾਂਸ: ਰੀਅਲ-ਟਾਈਮ ਵਿੱਚ ਫੰਕਸ਼ਨ ਕਾਲਾਂ ਨੂੰ ਕੈਪਚਰ ਅਤੇ ਲੌਗ ਕਰਨਾ
- HTML ਰੈਂਡਰਿੰਗ: ਏਜੰਟ ਦੇ ਜਵਾਬਾਂ ਨੂੰ ਸਜਾਏ ਹੋਏ ਟੇਬਲਾਂ ਅਤੇ ਇੰਟਰਐਕਟਿਵ ਡਿਸਪਲੇਜ਼ ਨਾਲ ਫਾਰਮੈਟ ਕਰਨਾ

## ਅਗਲੇ ਕਦਮ:
- ਵਾਧੂ MCP ਸਰਵਰਾਂ (ਮੌਸਮ, ਉਡਾਨਾਂ, ਰੈਸਟੋਰੈਂਟ) ਨੂੰ ਇੰਟੀਗ੍ਰੇਟ ਕਰੋ
- MCP ਅਤੇ A2A ਪ੍ਰੋਟੋਕੋਲ ਨੂੰ ਜੋੜ ਕੇ ਇੱਕ ਮਲਟੀ-ਏਜੰਟ ਸਿਸਟਮ ਬਣਾਓ
- ਆਪਣੇ ਡਾਟਾ ਸਰੋਤਾਂ ਲਈ ਕਸਟਮ MCP ਸਰਵਰ ਬਣਾਓ
- ਸੈਸ਼ਨ ਦੇ ਦੌਰਾਨ ਸਥਾਈ ਗੱਲਬਾਤ ਮੈਮਰੀ ਲਾਗੂ ਕਰੋ
- MCP ਸਰਵਰ ਓਰਕੇਸਟ੍ਰੇਸ਼ਨ ਨਾਲ ਏਜੰਟ ਨੂੰ Azure Functions 'ਤੇ ਡਿਪਲੌਇ ਕਰੋ
- ਯੂਜ਼ਰ ਪ੍ਰਮਾਣਿਕਤਾ ਅਤੇ ਬੁਕਿੰਗ ਸਮਰੱਥਾ ਸ਼ਾਮਲ ਕਰੋ



---

**ਅਸਵੀਕਰਤੀ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦ ਕੀਤਾ ਗਿਆ ਹੈ। ਜਦੋਂ ਕਿ ਅਸੀਂ ਸਹੀ ਹੋਣ ਦਾ ਯਤਨ ਕਰਦੇ ਹਾਂ, ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਦਿਓ ਕਿ ਸਵੈਚਾਲਿਤ ਅਨੁਵਾਦਾਂ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਸੁੱਚੀਤਤਾਵਾਂ ਹੋ ਸਕਦੀਆਂ ਹਨ। ਇਸ ਦੀ ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਮੌਜੂਦ ਮੂਲ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਪ੍ਰਮਾਣਿਕ ਸਰੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਨ ਜਾਣਕਾਰੀ ਲਈ, ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਇਸ ਅਨੁਵਾਦ ਦੀ ਵਰਤੋਂ ਤੋਂ ਪੈਦਾ ਹੋਣ ਵਾਲੇ ਕਿਸੇ ਵੀ ਗਲਤਫਹਿਮੀ ਜਾਂ ਗਲਤ ਵਿਆਖਿਆ ਲਈ ਅਸੀਂ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।  
